<a href="https://colab.research.google.com/github/taniae27/AprendizajeAutomatico/blob/main/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

# **Carga de datos**

Descarga el conjunto de datos de spam disponible en http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv y realiza lo siguiente:

In [2]:
df = pd.read_csv('http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv',header=None, sep=' ')
df

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
5168,0,0,0,0,3,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5169,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Conversion a numpy:

In [3]:
data = df.to_numpy()

X = data[:, :-1]
Y = data[:, -1]

# **Reporta el porcentaje de correos que están etiquetados como spam y como no spam en el conjunto de datos**

In [4]:
Detalle=pd.DataFrame({'Etiqueta':['Spam','No Spam'],'Registros':[Y.sum(),Y.shape[0]-Y.sum()],'Porcentaje':[(Y.sum()/Y.shape[0])*100,((Y.shape[0]-Y.sum())/Y.shape[0])*100]})
Detalle

,Etiqueta,Registros,Porcentaje
0,Spam,1500,29.00232
1,No Spam,3672,70.99768


In [5]:
print("El total de registros del conjunto de datos es de: {:}".format(Y.shape[0]))
print("El porcentaje de correos etiquetados como spam en el conjunto de datos es de: {:.2%}".format(Y.sum()/Y.shape[0]))
print("El porcentaje de correos etiquetados como no spam en el conjunto de datos es de: {:.2%}".format((Y.shape[0]-Y.sum())/Y.shape[0]))

El total de registros del conjunto de datos es de: 5172
El porcentaje de correos etiquetados como spam en el conjunto de datos es de: 29.00%
El porcentaje de correos etiquetados como no spam en el conjunto de datos es de: 71.00%


# **Divide aleatoriamente el conjunto de datos en el 60 % para entrenamiento, el 20 % para validación y el 20 % restante para prueba usando 0 como semilla para tu generador de números aleatorios**

In [6]:
# Se fija la semilla para el generador de números aleatorios
np.random.seed(0)

# Se crea un índice aleatorio para los datos
indices = np.random.permutation(len(X))

In [7]:
indices

array([3324,   15, 4950, ..., 1653, 2607, 2732])

In [8]:
# calcular los tamaños de cada conjunto
train_size = int(0.6 * len(X))
val_size = int(0.2 * len(X))
test_size = len(X) - train_size - val_size

In [9]:
print(train_size/len(X))
print(val_size/len(X))
print(test_size/len(X))

0.5999613302397525
0.19992266047950502
0.20011600928074247


In [10]:
# dividir los datos y etiquetas en los conjuntos
X_train = X[indices[:train_size]]
Y_train = Y[indices[:train_size]]
X_val = X[indices[train_size:train_size+val_size]]
Y_val = Y[indices[train_size:train_size+val_size]]
X_test = X[indices[train_size+val_size:]]
Y_test = Y[indices[train_size+val_size:]]

In [11]:
# comprobar el tamaño de los conjuntos
print("Entrenamiento:", "X_train= {:.2%}".format(X_train.shape[0]/X.shape[0]), "Y_train= {:.2%}".format(Y_train.shape[0]/Y.shape[0]))
print("Validación: ",  "X_val= {:.2%}".format(X_val.shape[0]/X.shape[0]), "Y_val= {:.2%}".format(Y_val.shape[0]/Y.shape[0]))
print("Prueba: ",  "X_test= {:.2%}".format(X_test.shape[0]/X.shape[0]), "Y_test= {:.2%}".format(Y_test.shape[0]/Y.shape[0]))

Entrenamiento: X_train= 60.00% Y_train= 60.00%
Validación:  X_val= 19.99% Y_val= 19.99%
Prueba:  X_test= 20.01% Y_test= 20.01%


# **Entrena 2 clasificadores bayesianos ingenuos con distintas distribuciones.**

In [12]:
mnb = MultinomialNB()
mnb.fit(X_train, Y_train)

MultinomialNB()

In [13]:
bnb = BernoulliNB()
bnb.fit(X_train, Y_train)

BernoulliNB()

In [14]:
preds_train_mnb = mnb.predict(X_train)
preds_val_mnb = mnb.predict(X_val)
preds_train_bnb = bnb.predict(X_train)
preds_val_bnb = bnb.predict(X_val)

In [20]:
accuracy_train_mnb = accuracy_score(Y_train, preds_train_mnb)
accuracy_val_mnb = accuracy_score(Y_val, preds_val_mnb)
accuracy_train_bnb = accuracy_score(Y_train, preds_train_bnb)
accuracy_val_bnb = accuracy_score(Y_val, preds_val_bnb)

In [23]:
vectorizer = CountVectorizer()


In [25]:
# Reportar el porcentaje de correos etiquetados como spam y no spam en el conjunto de datos
print("Porcentaje de correos etiquetados como spam en el conjunto de datos:", np.mean(Y))
print("Porcentaje de correos etiquetados como no spam en el conjunto de datos:", 1 - np.mean(Y))

# Reportar el porcentaje de predicciones correctas de cada clasificador en los conjuntos de entrenamiento y validación
print("Multinomial Naive Bayes - Entrenamiento:", accuracy_score(Y_train, preds_train_mnb))
print("Multinomial Naive Bayes - Validación:", accuracy_score(Y_val, preds_val_mnb))
print("Bernoulli Naive Bayes - Entrenamiento:", accuracy_score(Y_train, preds_train_bnb))
print("Bernoulli Naive Bayes - Validación:", accuracy_score(Y_val, preds_val_bnb))



Porcentaje de correos etiquetados como spam en el conjunto de datos: 0.2900232018561485
Porcentaje de correos etiquetados como no spam en el conjunto de datos: 0.7099767981438515
Multinomial Naive Bayes - Entrenamiento: 0.9526264904930712
Multinomial Naive Bayes - Validación: 0.9535783365570599
Bernoulli Naive Bayes - Entrenamiento: 0.9000966806316468
Bernoulli Naive Bayes - Validación: 0.9090909090909091
